In [ ]:
!pip install unstructured libmagic python-magic python-magic-bin
!pip install bs4
!pip install -qU langchain openai
!pip install -U langchain-community
!pip install langchain unstructured nltk
!pip install faiss-cpu
!pip install sentence_transformers
!pip install tiktoken
!pip install groq
!pip install -qU langchain-groq
!pip install langchain
!pip install langchain-huggingface
!pip install --upgrade nltk
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
!pip install spacy
!python -m spacy download en_core_web_sm

  Using cached libmagic-1.0.tar.gz (3.7 kB)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 61.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pickle
import os
import nltk
import langchain
import groq
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from  langchain.document_loaders import UnstructuredURLLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
article_title=[]
article_urls=[]

In [ ]:
url = "https://www.moneycontrol.com/news/business/stocks/"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
print('MoneyControl')
for article in soup.find_all('li', class_='clearfix'):
    title = article.find('a')['title']
    link = article.find('a')['href']
    article_title.append(title)
    article_urls.append(link)
    print("Title: ", title)
    print("Link: ", link)
print('-'*50)
print('CNBC')
url = "https://www.cnbctv18.com/stocks/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
articles = soup.find_all('a', class_='jsx-95506e352219bddb story-media')
for article in articles:
    title = article['title'].strip()
    link = urljoin(url, article['href'])
    article_title.append(title)
    article_urls.append(link)
    print("Title: ", title)
    print("Link: ", link)

MoneyControl
Title:  Market continues record run in second week; all sectors gain
Link:  https://www.moneycontrol.com/news/photos/business/markets/market-continues-record-run-in-second-week-all-sectors-gain-12831253.html
Title:  48 smallcaps give double-digit gain as broader indices climb to fresh high
Link:  https://www.moneycontrol.com/news/business/markets/48-smallcaps-give-double-digit-gain-as-broader-indices-climb-to-fresh-high-12831251.html
Title:  Dow posts record high close; S&P 500, Nasdaq end down
Link:  https://www.moneycontrol.com/news/business/markets/dow-posts-record-high-close-sp-500-nasdaq-end-down-12831252.html
Title:  Bulk deals: Morgan Stanley, Societe Generale, Citigroup buy 2.8% stake in Sterling and Wilson Renewable Energy
Link:  https://www.moneycontrol.com/news/business/markets/bulk-deals-morgan-stanley-societe-generale-citigroup-buy-2-8-stake-in-sterling-and-wilson-renewable-energy-12831175.html
Title:  FIIs net sell shares worth Rs 1,209 crore, DIIs net buy Rs

In [ ]:
article_title

['Market continues record run in second week; all sectors gain',
 '48 smallcaps give double-digit gain as broader indices climb to fresh high',
 'Dow posts record high close; S&P 500, Nasdaq end down',
 'Bulk deals: Morgan Stanley, Societe Generale, Citigroup buy 2.8% stake in Sterling and Wilson Renewable Energy',
 'FIIs net sell shares worth Rs 1,209 crore, DIIs net buy Rs 6,887-crore shares',
 "Trader's View: Early signs visible of sugar stocks turning sweet again",
 'Live: Nifty clocks a hattrick! 3rd straight week of gains | IT stocks in focus | Closing Bell',
 'Gainers & losers: Stocks and sectors that moved the most on September 27',
 'Taking Stock: Nifty below 26,200, Sensex falls 264 pts but gains 1% in a week',
 'Buy Tata Power Company; target of Rs 540: Sharekhan',
 "Brokerages cautious over PB Fintech's foray into healthcare sector",
 'Zee Media stock locked at 10% upper circuit as board approves fundraising Rs 200 crore via issue of warrants',
 'Wagers on stronger Indian r

In [ ]:
article_urls

['https://www.moneycontrol.com/news/photos/business/markets/market-continues-record-run-in-second-week-all-sectors-gain-12831253.html',
 'https://www.moneycontrol.com/news/business/markets/48-smallcaps-give-double-digit-gain-as-broader-indices-climb-to-fresh-high-12831251.html',
 'https://www.moneycontrol.com/news/business/markets/dow-posts-record-high-close-sp-500-nasdaq-end-down-12831252.html',
 'https://www.moneycontrol.com/news/business/markets/bulk-deals-morgan-stanley-societe-generale-citigroup-buy-2-8-stake-in-sterling-and-wilson-renewable-energy-12831175.html',
 'https://www.moneycontrol.com/news/business/markets/fiis-net-sell-shares-worth-rs-1209-crore-diis-net-buy-rs-6887-crore-shares-12831027.html',
 'https://www.moneycontrol.com/news/business/markets/traders-view-early-signs-visible-of-sugar-stocks-turning-sweet-again-12831036.html',
 'https://www.moneycontrol.com/news/videos/business/markets/live-nifty-clocks-a-hattrick-3rd-straight-week-of-gains-it-stocks-in-focus-closing

In [ ]:
os.environ['GROQ_API_KEY']='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'

In [ ]:
# try:
#     nltk.data.find('tokenizers/punkt')
# except LookupError:
#     nltk.download('punkt')

# data=[]
# for i in range(len(article_urls)):
#     url = article_urls[i]
#     loader = UnstructuredURLLoader(urls=[url])
#     try:
#       documents = loader.load()
#       for doc in documents:
#         data.append(doc)
#         print(doc.page_content)
#     except Exception as e:
#       print(f"Error loading URL: {e}")

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

loader = UnstructuredURLLoader(urls=['https://www.moneycontrol.com/news/business/stocks/buy-mahindra-and-mahindra-financial-services-target-of-rs-380-axis-capital-12827693.html'])
try:
  data = loader.load()
  print('Loaded')
except Exception as e:
  print(f"Error loading URL: {e}")



Loaded


In [ ]:
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0.9,
    max_tokens=500,
)


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs=text_splitter.split_documents(data)
len(docs)

8

In [ ]:
embeddings=HuggingFaceEmbeddings()
vector_index=FAISS.from_documents(docs,embeddings)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
chain=RetrievalQAWithSourcesChain.from_llm(llm=llm,retriever=vector_index.as_retriever())
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x78a7f366f790>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x78a7f32900d0>, temperature=0.9, model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=500), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_ty

In [ ]:
query="Give small summary"
langchain.debug=True
response=chain({"question":query},return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "Give small summary"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Trending Topics\n\nBusiness News LiveSwiggy IPOTencent Arkade Developers share priceGold Price Today\n\nBuy Mahindra and Mahindra Financial Services; target of Rs 380: Axis Capital\n\nAxis Capital is bullish on Mahindra and Mahindra Financial Services has recommended buy rating on the stock with a target price of Rs 380 in its research report dated September 19, 2024.\n\nBroker Research\n\nSeptember 23, 2024 / 12:51 IST\n\nBUY\n\nWatchlist\n\nPortfolio\n\nMessage\n\nSet Alert\n\nlive\n\nbselive\n\nnselive\n\nVolume\n\nTodays L/H\n\nMore\n\nAxis Capital's research report on Mahi

In [ ]:
answer = response["answer"]
answer = answer.replace("\n", "")
print(answer)

Axis Capital is bullish on Mahindra and Mahindra Financial Services with a buy rating and a target price of Rs 380, according to a research report dated September 19, 2024. The link to the article is provided. Additionally, the webpage contains various financial tools, resources, and information, as well as contact and support details, disclaimers, privacy policies, terms and conditions, and career opportunities.


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score , classification_report

analyzer=SentimentIntensityAnalyzer()

In [ ]:
df=pd.read_csv('all-data.csv',encoding='latin')
df.columns=['class','text']
df['text'] = df['text'].astype(str)
df

,class,text
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
...,...,...
4840,negative,LONDON MarketWatch -- Share prices ended lower...
4841,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4843,negative,Net sales of the Paper segment decreased to EU...


In [ ]:
df.duplicated().sum()

6

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.isnull().sum()

,0
class,0
text,0


In [ ]:
df['class'].value_counts()
df=df[~(df['class']=='neutral')]
df['class'].value_counts()

,count
class,
positive,1363
negative,604


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df['text'].values,df['class'].values,random_state=42,test_size=0.2)

In [ ]:
analyzer.polarity_scores(df['text'].values[4])

{'neg': 0.064, 'neu': 0.856, 'pos': 0.08, 'compound': 0.1531}

In [ ]:
df['vader_pred']=df['text'].apply(lambda x : analyzer.polarity_scores(x)['compound'])

<ipython-input-79-2c5b8a5e7dce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vader_pred']=df['text'].apply(lambda x : analyzer.polarity_scores(x)['compound'])


In [ ]:
df['pred_class']=df['vader_pred'].apply(lambda x: 'negative' if x <0 else 'positive' )

<ipython-input-80-1ea230deb47b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pred_class']=df['vader_pred'].apply(lambda x: 'negative' if x <0 else 'positive' )


In [ ]:
df

,class,text,vader_pred,pred_class
1,negative,The international electronic industry company ...,0.0000,positive
2,positive,With the new production plant the company woul...,0.8555,positive
3,positive,According to the company 's updated strategy f...,0.6705,positive
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,0.3485,positive
5,positive,"For the last quarter of 2010 , Componenta 's n...",0.1531,positive
...,...,...,...,...
4839,negative,HELSINKI Thomson Financial - Shares in Cargote...,0.6597,positive
4840,negative,LONDON MarketWatch -- Share prices ended lower...,-0.7269,negative
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...,0.7430,positive
4843,negative,Net sales of the Paper segment decreased to EU...,0.4404,positive


In [ ]:
print(classification_report(df['class'],df['pred_class']))

              precision    recall  f1-score   support

    negative       0.71      0.31      0.43       604
    positive       0.76      0.94      0.84      1363

    accuracy                           0.75      1967
   macro avg       0.73      0.63      0.64      1967
weighted avg       0.74      0.75      0.71      1967



In [ ]:
vader_score = analyzer.polarity_scores(answer)
predicted_class = 'negative' if vader_score['compound'] < 0 else 'positive'
print(f"VADER Sentiment Scores: {vader_score}")
print(f"Predicted Sentiment Class: {predicted_class}")

VADER Sentiment Scores: {'neg': 0.0, 'neu': 0.885, 'pos': 0.115, 'compound': 0.7506}
Predicted Sentiment Class: positive


In [ ]:
nse_stocks= pd.read_csv('/content/stocks&symb - Sheet1.csv')
nse_stocks

,NSE,Name,Symbol
0,NSE: 20MICRONS,20 Microns Limited,20MICRONS
1,NSE: 21STCENMGM,21st Century Management Services Limited,21STCENMGM
2,NSE: 3IINFOTECH,3i Infotech Limited,3IINFOTECH
3,NaN,3M India Limited,#REF!
4,NSE: 3PLAND,3P Land Holdings Limited,3PLAND
...,...,...,...
1808,NSE: ZOTA,Zota Health Care Limited,ZOTA
1809,NSE: ZUARI,Zuari Agro Chemicals Limited,ZUARI
1810,NSE: ZUARIGLOB,Zuari Global Limited,ZUARIGLOB
1811,NSE: ZYDUSWELL,Zydus Wellness Limited,ZYDUSWELL


In [ ]:
symbols= nse_stocks[['Name','Symbol']].copy()
# symbols['Name'] = symbols['Name'].astype(pd.StringDtype())
# symbols['Symbol'] = symbols['Symbol'].astype(pd.StringDtype())
symbols

,Name,Symbol
0,20 Microns Limited,20MICRONS
1,21st Century Management Services Limited,21STCENMGM
2,3i Infotech Limited,3IINFOTECH
3,3M India Limited,#REF!
4,3P Land Holdings Limited,3PLAND
...,...,...
1808,Zota Health Care Limited,ZOTA
1809,Zuari Agro Chemicals Limited,ZUARI
1810,Zuari Global Limited,ZUARIGLOB
1811,Zydus Wellness Limited,ZYDUSWELL


In [ ]:
query="Give me the names of company on which doc is based"
langchain.debug=True
comp=chain({"question":query},return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "Give me the names of company on which doc is based"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Mahindra & Mahindra - 23092024 - axis\n\nBroker Research\n\nTags: #Axis Capital #Buy #Mahindra and Mahindra Financial Services #Recommendations\n\nfirst published: Sep 23, 2024 12:51 pm\n\nTop Trends\n\nAngel TaxWiproBudget newsNew Income tax slabIPO News\n\nAdvertisement\n\nRemove Ad\n\nTrending news\n\n'They told me I was scaring customers': Man with facial condition asked to leave London restaurant by staff\n\n'Ghar jao': Swiggy CEO advises against hustle culture. 'Someone making sense,' says internet\n\n'We are a socialist...': Zerodha's Nith

In [ ]:
company_str = comp["answer"]
company_str = company_str.replace("\n", "")
print(company_str)

The names of the companies mentioned in the document are Mahindra & Mahindra and Mahindra and Mahindra Financial Services.


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(company_str)
stock_names = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
print(stock_names)


['Mahindra & Mahindra', 'Mahindra and Mahindra Financial Services']


In [ ]:
variations = []
for name in stock_names:
    if '&' in name:
        variations.append(name.replace('&', 'and'))
    elif 'and' in name:
        variations.append(name.replace('and', '&'))
stock_names.extend(variations)
print(stock_names)


['Mahindra & Mahindra', 'Mahindra and Mahindra Financial Services', 'Mahindra and Mahindra', 'Mahindra & Mahindra Financial Services']


In [ ]:
matching_symbols = symbols[symbols['Name'].apply(lambda x: any(name in x for name in stock_names))]['Symbol'].tolist()
print(matching_symbols)

['M&MFIN', 'M&M']
